<a href="https://colab.research.google.com/github/isikaykarakus/ForeoAIINternship/blob/main/Foreow1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install sentence-transformers faiss-cpu transformers pandas

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.5 MB/s eta 0:00:00


# **Tiny demo dataset (10 rows)**

In [ ]:
data = [
    {"phrase":"spill the tea","meaning":"share the gossip or secret","usage":"She spilled the tea about the new launch.","source_url":"https://example.com/urban/spill_the_tea"},
    {"phrase":"low-key","meaning":"subtly; not openly; a little bit","usage":"I’m low-key excited about this collab.","source_url":"https://example.com/urban/low_key"},
    {"phrase":"ghosting","meaning":"suddenly cutting off all communication","usage":"He stopped replying—total ghosting.","source_url":"https://example.com/urban/ghosting"},
    {"phrase":"stan","meaning":"an overzealous or obsessive fan","usage":"I stan that skincare brand.","source_url":"https://example.com/urban/stan"},
    {"phrase":"cap","meaning":"a lie; not true","usage":"They said the product is magic—sounds like cap.","source_url":"https://example.com/urban/cap"},
    {"phrase":"flex","meaning":"show off; boast","usage":"She flexed her skincare routine on IG.","source_url":"https://example.com/urban/flex"},
    {"phrase":"ratio","meaning":"a reply gets more likes than the original post (implies disagreement)","usage":"Their tweet got ratioed instantly.","source_url":"https://example.com/urban/ratio"},
    {"phrase":"simp","meaning":"someone who is overly attentive for affection/attention","usage":"Don’t simp for clout.","source_url":"https://example.com/urban/simp"},
    {"phrase":"yeet","meaning":"to throw with force; also an exclamation of excitement","usage":"Yeet that empty bottle in the bin!","source_url":"https://example.com/urban/yeet"},
    {"phrase":"mid","meaning":"average; not great","usage":"The results were mid tbh.","source_url":"https://example.com/urban/mid"},
]
df = pd.DataFrame(data)
df


,phrase,meaning,usage,source_url
0,spill the tea,share the gossip or secret,She spilled the tea about the new launch.,https://example.com/urban/spill_the_tea
1,low-key,subtly; not openly; a little bit,I’m low-key excited about this collab.,https://example.com/urban/low_key
2,ghosting,suddenly cutting off all communication,He stopped replying—total ghosting.,https://example.com/urban/ghosting
3,stan,an overzealous or obsessive fan,I stan that skincare brand.,https://example.com/urban/stan
4,cap,a lie; not true,They said the product is magic—sounds like cap.,https://example.com/urban/cap
5,flex,show off; boast,She flexed her skincare routine on IG.,https://example.com/urban/flex
6,ratio,a reply gets more likes than the original post...,Their tweet got ratioed instantly.,https://example.com/urban/ratio
7,simp,someone who is overly attentive for affection/...,Don’t simp for clout.,https://example.com/urban/simp
8,yeet,to throw with force; also an exclamation of ex...,Yeet that empty bottle in the bin!,https://example.com/urban/yeet
9,mid,average; not great,The results were mid tbh.,https://example.com/urban/mid


In [ ]:
# minimal normalisation
df["phrase_clean"] = df["phrase"].str.lower().str.strip()

# embed corpus text (phrase + meaning + usage)
corpus_texts = (df["phrase_clean"] + " — " + df["meaning"] + " — " + df["usage"]).tolist()
emb_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = emb_model.encode(corpus_texts, normalize_embeddings=True)

# FAISS index (cosine via inner product on normalised vectors)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# tiny generator
gen = pipeline("text2text-generation", model="google/flan-t5-small", max_new_tokens=120)

def search(query, k=3):
    q_emb = emb_model.encode([query], normalize_embeddings=True)
    D, I = index.search(q_emb, k)
    hits = df.iloc[I[0]].copy()
    hits["score"] = [float(s) for s in D[0]]
    return hits

def explain(query, k=3):
    hits = search(query, k)
    context = "\n".join([f"- {r.phrase}: {r.meaning} (e.g., {r.usage})" for _, r in hits.iterrows()])
    prompt = (
        f"Explain the slang '{query}' in clear, simple English. "
        f"Use the examples below for context. Add a short cultural note if helpful.\n\n{context}\n\nAnswer:"
    )
    out = gen(prompt)[0]["generated_text"]
    return hits[["phrase","meaning","usage","source_url","score"]], out

table, answer = explain("spill the tea", k=3)
display(table)
print("\n--- EXPLANATION ---\n", answer)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


,phrase,meaning,usage,source_url,score
0,spill the tea,share the gossip or secret,She spilled the tea about the new launch.,https://example.com/urban/spill_the_tea,0.680214
4,cap,a lie; not true,They said the product is magic—sounds like cap.,https://example.com/urban/cap,0.244999
8,yeet,to throw with force; also an exclamation of ex...,Yeet that empty bottle in the bin!,https://example.com/urban/yeet,0.225092



--- EXPLANATION ---
 spill the tea: share the gossip or secret (e.g., She spilled the tea about the new launch.)


In [ ]:
table, answer = explain("low-key", k=3)
display(table)
print("\n--- EXPLANATION ---\n", answer)


,phrase,meaning,usage,source_url,score
1,low-key,subtly; not openly; a little bit,I’m low-key excited about this collab.,https://example.com/urban/low_key,0.613782
9,mid,average; not great,The results were mid tbh.,https://example.com/urban/mid,0.117073
8,yeet,to throw with force; also an exclamation of ex...,Yeet that empty bottle in the bin!,https://example.com/urban/yeet,0.105683



--- EXPLANATION ---
 low-key: subtly; not openly; a little bit (e.g., I’m low-key excited about this collab.) - mid: average; not great (e.g., The results were mid tbh.)
